In [ ]:
import pandas as pd
import numpy as np
from pybnesian import hc, HeterogeneousBNType, GaussianNetwork, CLGNetwork, CLGNetworkType, KDENetworkType
from scipy.stats import rv_discrete
from drawdata import draw_scatter
import matplotlib.pyplot as plt
import math


In [ ]:
#draw_scatter()

In [ ]:
#df = pd.read_clipboard(sep=",")
#df.to_csv("toy-2class.csv", index = False)

In [ ]:
df = pd.read_csv("toy-3class.csv")

In [ ]:
df["class"] = df["z"].astype('category')
#df["z"] = df["z"].astype('category')
df = df.drop("z", axis = 1 )


learned = hc(df, bn_type = CLGNetworkType(), operators = ["arcs"], score = "validated-lik")
learned.num_arcs()

learned.fit(df)

In [ ]:
factor = learned.cpd("class")

In [ ]:
factor.parent_values()

In [ ]:
synt = learned.sample(1000).to_pandas()

In [ ]:
synt[synt["class"] == "c"].count()

In [ ]:
def euclidean_distance(x_cfx, x_og) :
    # Make sure attributes go in the same order
    #x_og = x_og[x_cfx.index]
    
    # Return Euclidean distance
    return np.sqrt(np.sum((x_cfx-x_og).values**2))

def delta_distance(x_cfx, x_og) :
    pass

def likelihood(x_cfx,bn) :
    class_cpd = bn.cpd("class")
    class_values = class_cpd.variable_values()
    cfx = x_cfx.copy()
    n_samples = x_cfx.shape[0]
    likelihood = 0
    for v in class_values :
        cfx["class"] = pd.Categorical([v]*n_samples, categories = class_values)
        likelihood = likelihood + math.e**bn.logl(cfx)
    return likelihood

def log_likelihood(x_cfx,bn) :
    return np.log(likelihood(x_cfx,bn))

def accuracy(x_cfx, y_og, bn) :
    class_cpd = bn.cpd("class")
    class_values = class_cpd.variable_values()
    cfx = x_cfx.copy()
    cfx["class"] = pd.Categorical([y_og], categories = class_values)
    prob = math.e**bn.logl(cfx)
    ll = likelihood(x_cfx,bn)
    if ll>0 :
        return prob/ll
    else :
        return 1


def straight_path(x_1, x_2) :
    spacing = int(euclidean_distance(x_1,x_2)/10)
    points = np.zeros(shape=(x_2.shape[1],spacing))
    for i,att in enumerate(x_2.columns) :
        points[i] = np.linspace(x_1[att].values[0],x_2[att].values[0],spacing)
    to_ret = pd.DataFrame(columns = x_2.columns, index = range(0,spacing))
    to_ret[:] = points.transpose()
    return to_ret

def path(x_1, x_2) :
    return straight_path(x_1,x_2)

def avg_path_logl(x_cfx, x_og, bn) :
    likelihood_path = (-log_likelihood(path(x_og, x_cfx),bn))**10
    return np.sum(likelihood_path)





In [ ]:
x_cfx = synt.iloc[[100]]
x_cfx = x_cfx.drop("class",axis = 1)
x_og = synt.iloc[[200]]
x_og = x_og.drop("class",axis = 1)
path_x = straight_path(x_cfx,x_og,learned)

In [ ]:
del path

In [ ]:
likelihood(path_x,learned)

In [ ]:
avg_path_likelihood(x_cfx,x_og,learned)

In [ ]:
likelihood(x_cfx,learned)

In [ ]:
euclidean_distance(synt.iloc[0].drop("class"), synt.iloc[2].drop("class"))

In [ ]:
synt.iloc[[0]]#.drop("class", axis = 1)

In [ ]:
learned.cpd("x")

In [ ]:
x_cfx = synt.iloc[[300]].copy()
#x_cfx = x_cfx.drop("class",axis = 1)
x_cfx["x"] = [192.069]
x_cfx.loc[x_cfx.index]["y"] = 444.152 + -1.215*192.069
x_cfx.loc[x_cfx.index]["class"] = pd.Categorical(["a"], categories = ["a","b","c"])
print(x_cfx)

In [ ]:
print(math.e**learned.cpd("x").logl(x_cfx.drop(["y"], axis = 1)))
print(math.e**learned.cpd("y").logl(x_cfx))
print(math.e**learned.cpd("class").logl(x_cfx.drop(["x","y"], axis = 1)))

In [ ]:
from scipy.stats import norm

In [ ]:
x = norm(192.069, np.sqrt(2554.987))
y = norm(444.152 + -1.215*192.069, np.sqrt(16335.215))

In [ ]:
x.pdf(192.069)*y.pdf(444.152 + -1.215*192.069)*0.479

In [ ]:
x.pdf(192.069)

In [ ]:
math.e**learned.logl(x_cfx)

In [ ]:
likelihood(x_cfx,learned)

In [ ]:
factor.parent_values()

In [ ]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize

In [ ]:
class MyProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=2,
                         n_obj=3,
                         n_ieq_constr=2,
                         xl=np.array([0]*2),
                         xu=np.array([700]*2))

    def _evaluate(self, x, out, *args, **kwargs):
        x_cfx = x_og.copy()
        x_cfx[:] = x
        f1 = euclidean_distance(x_cfx,x_og)
        f2 = -log_likelihood(x_cfx, learned)
        f3 = avg_path_logl(x_cfx, x_og, learned)
        out["F"] = np.column_stack([f1, f2, f3])
        
        g1 = -0.1#-likelihood(x_cfx, learned)+0.0000001
        g2 = accuracy(x_cfx,y_og,learned)-0.05
        out["G"] = np.column_stack([g1,g2])

In [ ]:
problem = MyProblem()

x_og = synt.iloc[[300]].copy()
x_og = x_og.drop("class",axis = 1)
x_og.columns = ["x","y"]
x_og["x"] = 192.069
x_og["y"] = 444.152 + -1.215*192.069 + np.sqrt(16335.215)*2
y_og = "a"
#x_cfx["class"] = pd.Categorical("a", categories = ["a","b","c"])

algorithm = NSGA2(pop_size=100)

res = minimize(problem,
               algorithm,
               ('n_gen', 10),
               seed=1,
               verbose=True)

# calculate a hash to show that all executions end with the same result
print("hash", res.F.sum())

In [ ]:
x_og

In [ ]:
to_plot = df.drop("class", axis = 1)
colours = df["class"].to_numpy()
colours[colours == "a"] = "green"
colours[colours == "b"] = "yellow"
colours[colours == "c"] = "blue"
plt.scatter(to_plot["x"],to_plot["y"], color = colours)
plt.scatter(res.X.transpose()[0],res.X.transpose()[1],color = "red")
plt.scatter(x_og["x"],x_og["y"], color = "black")
plt.show()

In [ ]:
x_test = x_og.copy()
x_test.x = 450.0
x_test.y = 425.0
print("Logl:",log_likelihood(x_test,learned))
print("Path logl:",avg_path_logl(x_test,x_og,learned))
print("Accuracy:",accuracy(x_test,"a",learned))

In [ ]:
0.9**2

In [ ]:
x_test = x_og.copy()
x_test.x = 700.0
x_test.y = 700.0
print("Logl:",log_likelihood(x_test,learned))
print("Path logl:",avg_path_logl(x_test,x_og,learned))
print("Accuracy:",accuracy(x_test,"a",learned))

In [ ]:
x = [95, 102.5]
y = [5, 17]

x_new = 100

y_new = np.interp(x_new, x, y)
print(y_new)
# 13.0

plt.plot(x, y, "og-", x_new, y_new, "or");

In [ ]:
x = [95, 102.5]
y = [5, 17]

x_new = np.arange(95,102.5,0.5)

y_new = np.interp(x_new, x, y)
print(y_new)
# 13.0

#plt.plot(x, y, "og-", x_new, y_new, "or");

In [ ]:
np.meshgrid(np.arange(-5, 5.1, 0.5), np.arange(-5, 5.1, 0.5), np.arange(-5, 5.1, 0.5))

In [ ]:
1.1**0.3

In [ ]:
x = np.arange(0.01,1,0.01)
z=1
exp = 1
f_x = (-np.emath.logn(10,x)+z)**exp
plt.plot(x,f_x)
plt.show()

In [ ]:
plt.plot(x[:-1],np.diff(f_x))

In [ ]:
plt.plot(x,f_x-x)

In [ ]:
0.9**2